In [181]:
from IPython.display import display, clear_output
import bokeh
import mibian
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import scipy
import math
from scipy import stats
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, push_notebook
import ipywidgets as widgets
from ipywidgets import FloatSlider
output_notebook()

Loading BokehJS ...

In [150]:
def simulation(b):
    year_day_count = 250
    num_interval = year_day_count
    num_sample = 1000

    iv = 0.30
    rv = 0.28
    stock_initial_price = 100.0

    # Initial Straddle Cost
    # Mibian package usage: BS([underlyingPrice, strikePrice, interestRate, daysToExpiration], volatility=x)
    option = mibian.BS([stock_initial_price, stock_initial_price, 0, year_day_count * 365/250], volatility=iv*100)
    straddle_price = option.callPrice + option.putPrice

    #Simulation
    profit_list = np.zeros(num_sample)
    price_path_list = []
    # Simulate flat Dirichlet Distribution random numbers bewteen 0 and 1 such that they sum up to 1
    dirichlet_random_num = np.random.dirichlet(alpha=((0.01),) * num_interval, size=num_sample)
    print('Simulating Number of Path = {}'.format(num_sample))
    for i, variance_path in (enumerate(tqdm(dirichlet_random_num))):
        # Assign daily return path (unsigned) to match total variance corresponding to 28% realized vol
        return_path = np.sqrt(variance_path * (rv ** 2))
        # Randomly assign Positive or Negative return each day
        sign_path = random.choices(population=[-1, 1], k=num_interval)
        return_path = np.append(np.array([0]), return_path * sign_path)
        # Get stock price path from cumulative return products
        price_path = stock_initial_price * np.cumprod(1 + return_path)
        stock_ending_price = price_path[-1]
        option_object_list = [mibian.BS([stock_price, stock_initial_price, 0, (year_day_count-d) * 365/250], volatility=iv*100) if year_day_count-d != 0 else None for d, stock_price in enumerate(price_path)]
        # Calculate Delta each day
        delta_list = np.array([-(option_object.callDelta + option_object.putDelta) if option_object is not None else 0 for option_object in option_object_list])
        # Calculate Delta trade qty each day
        delta_trade_list = -np.append(delta_list[0:1], np.diff(delta_list))
        price_path_list.append(price_path)
        # Final profit calculation = Straddle Price - drift away from ATM + all delta hedging pnl
        profit = straddle_price - abs(stock_initial_price - stock_ending_price) + np.sum((stock_ending_price - price_path) * delta_trade_list)
        profit_list[i] = profit
    print('Percentage of time (PnL < 0): {0:.2f}%'.format(len(profit_list[profit_list < 0]) / len(profit_list) * 100))
    print('Average PnL: {0:.2f}'.format(np.mean(profit_list)))
    print('Median PnL: {0:.2f}'.format(np.median(profit_list)))
    print('Highest PnL: {0:.2f}'.format(np.max(profit_list)))
    print('Lowest PnL: {0:.2f}'.format(np.min(profit_list)))
    print('Select Path to display based on PnL Percentile')
    percentile_slider = widgets.IntSlider(
        value=50,
        min=0,
        max=100,
        step=1,
        description='Percentile',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
    )
    display(percentile_slider)
    def on_value_change(value):
        percentile = value['new']
        i_near = abs(profit_list-np.percentile(profit_list, percentile, interpolation='nearest')).argmin()
        price_path = price_path_list[i_near]

        p = figure(plot_width = 800, plot_height = 300, title = 'Simulated Stock Price Path corresponding to {}% PnL percentile'.format(percentile), 
                   x_axis_label = 'Days to Expiry', y_axis_label = 'Stock Price', id='stock_price_path')

        p.line(x=range(year_day_count+1)[::-1], y=price_path, color='red', legend_label='Ending PnL = {}'.format(profit_list[i_near].round(2)))
        p.x_range.flipped = True
        
        h = show(p, notebook_handle=True)
        push_notebook(h)

    percentile_slider.observe(on_value_change, names='value')

In [201]:
SPY_price = pd.read_csv('SPY.csv')['Adj Close']
SPY_return = np.diff(np.log(SPY_price))
SPY_vol = np.sqrt((SPY_return ** 2) * 250)
np.std(SPY_vol)

0.08062530983764232

In [177]:
variance_path = np.random.dirichlet(alpha=((0.01),) * num_interval, size=num_sample)
return_path = np.sqrt(variance_path * (rv ** 2))
print(np.std(np.sqrt((return_path**2) * 250)))

0.2750887066266743


In [205]:
variance_path = np.random.dirichlet(alpha=((1),) * num_interval, size=num_sample)
return_path = np.sqrt(variance_path * (rv ** 2))
print(np.std(np.sqrt((return_path**2) * 250)))

0.1290137025876481


In [198]:
variance_path = np.random.dirichlet(alpha=((3),) * num_interval, size=num_sample)
return_path = np.sqrt(variance_path * (rv ** 2))
print(np.std(np.sqrt((return_path**2) * 250)))

0.07825980811896886


In [179]:
variance_path = np.random.dirichlet(alpha=((100),) * num_interval, size=num_sample)
return_path = np.sqrt(variance_path * (rv ** 2))
print(np.std(np.sqrt((return_path**2) * 250)))

0.013838316192845312


In [151]:
def display_simulation():
    button = widgets.Button(
        description='Run Simulations',
        disabled=False,
        button_style='success',
    )
    display(button)
    button.on_click(simulation)
display_simulation()

Button(button_style='success', description='Run Simulations', style=ButtonStyle())

Simulating Number of Path = 1000



Percentage of time (PnL < 0): 40.60%
Average PnL: 0.65
Median PnL: 0.61
Highest PnL: 10.82
Lowest PnL: -22.54
Select Path to display based on PnL Percentile


IntSlider(value=50, continuous_update=False, description='Percentile')

In [149]:
def display_simulation():
    button = widgets.Button(
        description='Run Simulations',
        disabled=False,
        button_style='success',
    )
    display(button)
    button.on_click(simulation)
display_simulation()

Button(button_style='success', description='Run Simulations', style=ButtonStyle())

Simulating Number of Path = 1000



Percentage of time (PnL < 0): 0.00%
Average PnL: 1.59
Median PnL: 1.58
Highest PnL: 3.84
Lowest PnL: 0.26
Select Path to display based on PnL Percentile


IntSlider(value=50, continuous_update=False, description='Percentile')